In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

%matplotlib inline

sns.set_style('whitegrid')

# **Data Input**

In [ ]:
train=pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')
subm=pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv')

In [ ]:
train.head(5)

**We have imported models for regression**

In [ ]:
X_train=train.drop(['id','loss'],axis=1)
X_subm=subm.drop(['id'],axis=1)
y=train[['loss']]

**Train Test split**

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X_train,y,random_state=42,test_size=0.2)
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,random_state=42,test_size=0.2)

**Checking the need to transform**

In [ ]:
# #MinMaxScaler
s1=MinMaxScaler()
x_train=s1.fit_transform(x_train)
x_val=s1.transform(x_val)
x_test=s1.transform(x_test)
x_subm=s1.transform(X_subm)

In [ ]:
#StandardScaler
# s1=StandardScaler()
# x_train=s1.fit_transform(x_train)
# x_val=s1.transform(x_val)
# x_test=s1.transform(x_test)
# x_subm=s1.transform(X_subm)

**Regression Model (Tensorflow model)**

In [ ]:
model=Sequential()

model.add(Dense(x_train.shape[1],activation='relu'))
model.add(Dense(2*x_train.shape[1],activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2*x_train.shape[1],activation='relu'))
model.add(Dense(x_train.shape[1],activation='relu'))

model.add(Dense(1,activation='relu'))

model.compile(optimizer='adam',metrics=[tf.keras.metrics.RootMeanSquaredError()],loss='mse')


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
e1=EarlyStopping(verbose=1)#restore_best_weights=True)

In [ ]:
history=model.fit(x=x_train,y=y_train,epochs=50,callbacks=[e1],workers=5,validation_data=(x_val,y_val))
pred1=model.predict(x_test)

In [ ]:
d1=pd.DataFrame(history.history)
# d1.iloc[:,[1,3]]
d1.iloc[:,[1,3]].plot(figsize=(8,8))

In [ ]:
from sklearn.metrics import mean_squared_error

print(f"RMSE :{mean_squared_error(y_test,pred1,squared=False)}")

In [ ]:
model.summary()

**Output**

In [ ]:
prediction=model.predict(x_subm)
sub=pd.read_csv('../input/tabular-playground-series-aug-2021/sample_submission.csv')
sub['loss']=prediction
sub.to_csv(f'vote.csv',index=False)